In [ ]:
import torch
import torchvision
import time
from torch import nn
from torch import optim
from torch.utils import data
from Validate import validate_net
from Test import test_net
from misc import print_metrics, training_curve 
from PIL import Image
import os
import re
import argparse
from collections import defaultdict
import numpy as np
import logging
import csv
from torchvision import transforms, datasets, models
import sklearn.metrics as mtc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
###########################
# Checking if GPU is used
###########################

use_cuda=torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()
device=torch.device("cuda:0" if use_cuda else "mps" if use_mps else "cpu")
device

In [ ]:
batch_size=32
test_root_dir= "path to your root directory" # Example: "../../Project/GastroVision22/test/"
model_evaluation_directory = "Path to your all selected model directory" # Example: "../selected_models/"
greedy_soup_file = "Name to your greedy soup result file" # Example: "greedy_soups_sorted_result_" + str(batch_size)
uniform_soup_file = "Name to your uniform soup result file" # Example: "uniform_soups_sorted_result_" + str(batch_size)

In [ ]:
####################################
# Training
####################################

trans={
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.4762, 0.3054, 0.2368],
                             [0.3345, 0.2407, 0.2164])
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.4762, 0.3054, 0.2368],
                             [0.3345, 0.2407, 0.2164])
    ]),
    
    # Test does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.4762, 0.3054, 0.2368],
                             [0.3345, 0.2407, 0.2164])
    ]),
}

# Model Soups

## Uniform Model Soups

In [ ]:
import inspect
import os
import sys
import time
import numpy as np

def uniform_soup(model, paths, device = "cpu", by_name = False):
    try:
        import torch
    except:
        print("If you want to use 'Model Soup for Torch', please install 'torch'")
        return model
        
    if not isinstance(paths, list):
        paths = [paths]
    model = model.to(device)
    model_dict = model.state_dict()
    soups = {key:[] for key in model_dict}
    for i, model_path in enumerate(paths):
        if ".DS_Store" in model_path: # only for Mac users
            print("Cannot process ", model_path)
            print("Continue...")
            continue
        print("Loading: ", model_path)
        checkpoint=torch.load(model_path,map_location=device)   # loading model
        # change you dictionary name according to your saved model name
        weight_dict = checkpoint['model_state_dict']
        if by_name:
            weight_dict = {k:v for k, v in weight_dict.items() if k in model_dict}
        for k, v in weight_dict.items():
            soups[k].append(v)
    if 0 < len(soups):
        soups = {k:(torch.sum(torch.stack(v), axis = 0) / len(v)).type(v[0].dtype) for k, v in soups.items() if len(v) != 0}
        model_dict.update(soups)
        model.load_state_dict(model_dict)
    return model

In [ ]:
# modifying order of model paths
model_names = os.listdir(model_evaluation_directory)
# Add all your model weights in sorted order for greedy soup implementation
model_names = ["C_22_32.pth", "C_32_64.pth", "C_23_64.pth", "C_29_128.pth", "C_13_32.pth", "C_27_32.pth", "C_23_128.pth", "C_44_64.pth", "C_32_128.pth"]

In [ ]:
#place all the model that are going to apply in model soup
model_paths = []
for name in model_names:
    model_paths.append(model_evaluation_directory + name)
    
model_paths

### Creating model again

In [ ]:
## recreating model
n_classes=22  # number of classes used for training
#Initialize model
best_model = torchvision.models.densenet121(weights=True).to(device)   # make weights=True if you want to download pre-trained weights


# Option to freeze model weights
for param in best_model.parameters():
    param.requires_grad = True
    
n_inputs = best_model.classifier.in_features
best_model.classifier = nn.Sequential(
              nn.Linear(n_inputs, n_classes),                  
              nn.LogSoftmax(dim=1))


best_model.to(device)

In [ ]:
uniform_model = uniform_soup(best_model, model_paths)
uniform_model

In [ ]:
#creating testing data
print('Generating test data')
#Generators
test_dataset= datasets.ImageFolder(test_root_dir,transform=trans['test'])
test_generator=data.DataLoader(test_dataset,batch_size)
criterion = nn.NLLLoss()

In [ ]:
def test_model(model, filename):

    ############################
    #     Test uniform model
    ############################
    test_list=[]
    model.to(device)
    model.eval()
    print("Evaluating model")
    with torch.no_grad():
           test_loss, test_metrics, test_num_steps=test_net(model,test_generator,device,criterion)

    print_metrics(test_metrics,test_num_steps)
    test_list.append(test_loss)


    for k, vl in test_metrics.items():      
        test_list.append(vl)              # append metrics results in a list

    ##################################################################
    # Writing test results to a csv file 
    ##################################################################

    key_name=['Test_loss','Test_micro_precision','Test_micro_recall','Test_micro_f1','Test_macro_precision','Test_macro_recall','Test_macro_f1','Test_mcc']
    try:

            with open(filename+str('.csv'), 'a',newline="") as f:
                wr = csv.writer(f,delimiter=",")
                wr.writerow(key_name)
                zip(test_list)
                wr.writerow(test_list) 
                wr.writerow("") 
    except IOError:
            print("I/O Error")  
            
    return test_metrics

In [ ]:
test_model(uniform_model, uniform_soup_file)

## Greedy Soups

In [ ]:
def metric(y_true, y_pred):
    return ((y_true == y_pred.argmax(axis = -1)).sum() / len(y_true)).to("cpu").item()

def greedy_soup(model, path, data, metric, device = "cpu", update_greedy = False, compare = np.greater_equal, by_name = False, digits = 4, verbose = True, y_true = "y_true"):
    try:
        import torch
    except:
        print("If you want to use 'Model Soup for Torch', please install 'torch'")
        return model

    if not isinstance(path, list):
        path = [path]
    score, soup = None, []
    model = model.to(device)
    model.eval()
    model_dict = model.state_dict()
    input_key = [key for key in inspect.getfullargspec(model.forward).args if key !=  "self"]
    input_cnt = len(input_key)
    for i, model_path in enumerate(path):
        if ".DS_Store" in model_path:
            print("Cannot process ", model_path)
            print("Continue...")
            continue
        if update_greedy:
            checkpoint=torch.load(model_path,map_location=device)   # loading model
            weight_dict = checkpoint['model_state_dict']
            if by_name:
                weight_dict = {k:v for k, v in weight_dict.items() if k in model_dict}
            if len(soup) != 0:
                weight_dict = {key:(torch.sum(torch.stack([weight_dict[key], soup[key]]), axis = 0) / 2).type(weight_dict[key].dtype)  for key in model_dict.keys()}
            model_dict.update(weight_dict)
            model.load_state_dict(model_dict)
        else:
            model = uniform_soup(model, soup + [model_path], device = device, by_name = by_name)
                
        iterator = iter(data)
        history = []
        step = 0
        start_time = time.time()
        while True:
            try:
                text = ""
                iter_data = next(iterator)
                if not isinstance(iter_data, dict):
                    x = iter_data[:input_cnt]
                    y = list(iter_data[input_cnt:])
                    y = [d.to(device) if isinstance(d, torch.Tensor) else d for d in y]
                    d_cnt = len(y[0])
                else:
                    x = [iter_data[k] for k in input_key if k in iter_data]
                x = [d.to(device) if isinstance(d, torch.Tensor) else d for d in x]
                step += 1
                #del x

                with torch.no_grad():
                    logits = model(*x)
                    if isinstance(logits, torch.Tensor):
                        logits = [logits]
                        
                    if isinstance(iter_data, dict):
                        metric_key = [key for key in inspect.getfullargspec(func).args if key !=  "self"]
                        if len(metric_key) == 0:
                            metric_key = [y_true]
                        y = [iter_data[k] for k in metric_key if k in iter_data]
                        y = [d.to(device) if isinstance(d, torch.Tensor) else d for d in y]
                        d_cnt = len(y[0])
                    metric_val = np.array(metric(*(y + logits)))
                    if np.ndim(metric_val) == 0:
                        metric_val = [float(metric_val)] * d_cnt
                    history += list(metric_val)
                    #del y, logits

                if verbose:
                    sys.stdout.write("\r[{name}] step: {step} - time: {time:.2f}s - {key}: {val:.{digits}f}".format(name = os.path.basename(model_path), step = step, time = (time.time() - start_time), key = metric.__name__ if hasattr(metric, "__name__") else str(metric), val = np.nanmean(history), digits = digits))
                    sys.stdout.flush()
            except StopIteration:
                print("")
                #gc.collect()
                break
        if 0 < len(history) and (score is None or compare(np.nanmean(history), score)):
            score = np.nanmean(history)
            if update_greedy:
                soup = weight_dict
            else:
                soup += [model_path]
    if len(soup) != 0:
        if update_greedy:
            model_dict.update(soup)
            model.load_state_dict(model_dict)
        else:
            model = uniform_soup(model, soup, device = device, by_name = by_name)
        if verbose:
            print("greedy soup best score : {val:.{digits}f}".format(val = score, digits = digits))
    return model

In [ ]:
greedy_model = greedy_soup(model=best_model, path=model_paths, data=test_generator, metric=metric, device = "cpu", update_greedy = False, compare = np.greater_equal, by_name = False, digits = 4, verbose = True, y_true = "y_true")

greedy_model

In [ ]:
test_model(greedy_model, greedy_soup_file)

### Greedy Updated Weights

In [ ]:
greedy_model_update = greedy_soup(model=best_model, path=model_paths, data=test_generator, metric=metric, device = "cpu", update_greedy = True, compare = np.greater_equal, by_name = False, digits = 4, verbose = True, y_true = "y_true")

greedy_model_update